In [1]:
import pandas as pd
print(pd.__version__)

2.2.3


In [2]:
import os
import sys
# Add the root directory to Python path
root_dir = "C:\\Users\\Jordan Lankford\\Documents\\GitHub\\FineTune-DQN"
if root_dir not in sys.path:
    sys.path.append(root_dir)

# Now imports should work
from src.utils.utils import set_seeds
import torch
import torch.nn as nn
import gymnasium as gym
import numpy as np
from src.utils.utils import set_seeds
from torchsummary import summary

def encode_state(obs, n_states):
    obs = torch.Tensor(obs)
    return nn.functional.one_hot(obs.long(), n_states).float()

def get_mask(info, n_actions=25):
    allowed_actions = info['admissible_actions']
    mask = np.zeros(n_actions)
    mask[allowed_actions] = 1
    return torch.Tensor(mask).unsqueeze(0)

# Modified QNetwork to match the deep DQN structure
class QNetwork(nn.Module):
    def __init__(self, env):
        super().__init__()
        self.n_states = env.observation_space.n
        self.n_actions = env.action_space.n
        
        # Match the architecture of the deep DQN
        self.network = nn.Sequential(
            nn.Linear(self.n_states, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, self.n_actions)
        )

    def forward(self, x, action_masks=None):
        q_values = self.network(x)
        if action_masks is not None:
            q_values = q_values - ((1 - action_masks) * 1e10)
        return q_values

# Create environment and network
env = gym.make('Sepsis/ICU-Sepsis-v2')
q_network = QNetwork(env)

# Load the trained deep DQN weights
model_path = r"C:\Users\Jordan Lankford\Documents\GitHub\FineTune-DQN\models\dqn\dqn_final_seed_0.pt"
state_dict = torch.load(model_path)

# Print the keys in the state dict to verify structure
print("State dict keys:", state_dict.keys())

# Load the weights
try:
    q_network.load_state_dict(state_dict)
    print("Successfully loaded weights!")
except Exception as e:
    print(f"Error loading weights: {e}")
    # Try loading with strict=False if there are issues
    q_network.load_state_dict(state_dict, strict=False)
    print("Loaded weights with strict=False")

# Print model structure
print("\nModel Architecture:")
print(q_network)


State dict keys: odict_keys(['network.0.weight', 'network.0.bias', 'network.2.weight', 'network.2.bias', 'network.4.weight', 'network.4.bias'])
Successfully loaded weights!

Model Architecture:
QNetwork(
  (network): Sequential(
    (0): Linear(in_features=716, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=25, bias=True)
  )
)


C:\Users\Jordan Lankford\AppData\Local\Temp\ipykernel_8356\2027990685.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


In [3]:
weight_shapes = []
for param in q_network.parameters():
    weight_shapes.append(param.shape)
print(weight_shapes)


[torch.Size([128, 716]), torch.Size([128]), torch.Size([128, 128]), torch.Size([128]), torch.Size([25, 128]), torch.Size([25])]


In [4]:
values_in_dimension=[]
for shape in weight_shapes:
    values_in_dimension.append(np.prod(shape))
print(values_in_dimension)

[np.int64(91648), np.int64(128), np.int64(16384), np.int64(128), np.int64(3200), np.int64(25)]


In [5]:
weights = [param.detach().numpy() for param in q_network.parameters()]  # Get all model weights as numpy arrays


In [6]:
def f(fly, flies_history=None, model=q_network, env=env, eval_seeds=[0, 1, 2, 3, 4]):
    """Use fixed seeds for more consistent evaluation while maintaining some variety"""
    cloned_model = q_network
    reshaped_fly = reshape_to_model_weights(fly, model)
    cloned_model.load_state_dict(dict(zip([name for name, _ in cloned_model.named_parameters()], reshaped_fly)))
    cloned_model.eval()

    episode_rewards = []
    episodes_per_seed = 4000  # 20000 total episodes divided among 5 seeds
    
    for seed in eval_seeds:
        env.reset(seed=seed)  # Set environment seed
        
        for episode in range(episodes_per_seed):
            state, info = env.reset()
            episode_reward = 0
            done = False
            
            while not done:
                state_encoded = encode_state(np.array([state]), env.observation_space.n)
                action_mask = get_mask(info, n_actions=env.action_space.n)
                
                with torch.no_grad():
                    q_values = cloned_model(state_encoded, action_mask)
                    action = torch.argmax(q_values).item()
                
                next_state, reward, terminated, truncated, info = env.step(action)
                episode_reward += reward
                state = next_state
                done = terminated or truncated

            episode_rewards.append(episode_reward)

    fitness = np.mean(episode_rewards)
    return -fitness
    #return -f_avrg(fitness, flies_history) if flies_history is not None else -fitness

In [7]:
#run fly through the fitness function 20 times, calculate the range in fitness values

#f(fly, flies_history = None, model=q_network,env=env)
fitness_values = []
for i in range(10):
    fitness_values.append(f(flies[0], flies_history = None, model=q_network,env=env))

print(fitness_values)
#range of fitness values
range_fitness = max(fitness_values) - min(fitness_values)
print(range_fitness)


NameError: name 'flies' is not defined

In [ ]:
#standard deviation of fitness values
std_fitness = np.std(fitness_values)
print(std_fitness)
#mean of fitness values
mean_fitness = np.mean(fitness_values)
print(mean_fitness)




In [ ]:
for i in range(20):
    #take the firtst ith number of flies and caluculate mean fitness
    mean_fitness = np.mean(fitness_values[:i])
    print("mean when using", i ,"flies:", mean_fitness)

In [8]:
def reshape_to_model_weights(flattened_fly, model=q_network):
    # List to store reshaped weights
    reshaped_weights = []
    counter = 0  # Counter for elements in fly
    
    # Get the shapes of the model's parameters
    values_in_dimension = [param.numel() for param in model.parameters()]
    weight_shapes = [param.shape for param in model.parameters()]
    
    for i in range(len(values_in_dimension)):
        # Get the flattened weights for this layer
        weight_flat = flattened_fly[counter: counter + values_in_dimension[i]]
        
        # Reshape and convert to torch.Tensor
        reshaped_layer_weights = torch.tensor(weight_flat, dtype=torch.float32).reshape(weight_shapes[i])
        
        # Append the reshaped weight to the list
        reshaped_weights.append(reshaped_layer_weights)
        
        # Move the counter to the next parameter block
        counter += values_in_dimension[i]
    
    return reshaped_weights


In [9]:
def initialize_flies_from_model(number_of_flies, weights, model, inclusive):
    population = []

    if inclusive == False:
        for i in range(number_of_flies):
            fly = []  # This will hold the flattened weights for a single fly
            
            for param in model.parameters():  # Iterate through model parameters (weights and biases)
                # Generate random weights with the same shape as the model's weight matrix
                scalers = np.random.uniform(0.9, 1.1, size=param.shape)
                print(param.shape)  # Print the shape to see what the weights look like
                scaledweights = param.detach().numpy() * scalers  # Multiply the weight values by the scalers
                
                # Flatten the random weights and add them to the fly
                flattened_weights = scaledweights.flatten()
                fly.append(flattened_weights)
            
            # After flattening each layer's weights, flatten the entire fly and add to population
            population.append(np.concatenate(fly))  # Concatenate the list of flattened weight arrays
            
    if inclusive == True:
        for i in range(number_of_flies):
            fly = []  # This will hold the flattened weights for a single fly
            
            for param in model.parameters():  # Iterate through model parameters (weights and biases)
                if i == 0:
                    flattened_weights = param.detach().numpy().flatten()  # Get the flattened weights for the first fly
                    fly.append(flattened_weights)
                else:
                    scalers = np.random.uniform(0.9, 1.1, size=param.shape)
                    print(param.shape)  # Print the shape to see what the weights look like
                    scaledweights = param.detach().numpy() * scalers  # Multiply the weight values by the scalers
                    
                    # Flatten the random weights and add them to the fly
                    flattened_weights = scaledweights.flatten()
                    fly.append(flattened_weights)
                
            # After flattening each layer's weights, flatten the entire fly and add to population
            population.append(np.concatenate(fly))  # Concatenate the list of flattened weight arrays
        
    return population


In [10]:
num_of_flies = 10

flies = initialize_flies_from_model(num_of_flies,weights,q_network,True)



from collections import deque
max_history = 5
flies_history = [deque(maxlen=max_history) for _ in range(num_of_flies)]

torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Size([128])
torch.Size([128, 128])
torch.Size([128])
torch.Size([25, 128])
torch.Size([25])
torch.Size([128, 716])
torch.Siz

In [ ]:
print(flies_history[0])

In [11]:
def f_avrg(fitness, flies_history):

    flies_history.append(fitness)

    print(flies_history)

    return sum(flies_history) / len(flies_history) # mean
    

In [ ]:
f(flies[0],flies_history[0])

In [ ]:
len(flies[0])

In [12]:
N = len(flies)			# POPULATION SIZE
D = len(flies[0])					# DIMENSIONALITY 
#delta = 0.005			# DISTURBANCE THRESHOLD 
maxIterations = 200	# ITERATIONS ALLOWED
sd = np.std(flies[0])		# STANDARD DEVIATION

In [13]:
#print the min and max values and the range of the initial population
min_value = np.min(flies[0])
max_value = np.max(flies[0])
range_value = max_value - min_value
print("Initial Population:")
print("Min:", min_value)
print("Max:", max_value)
print("Range:", range_value)
sd = np.std(flies[0])
print("Standard Deviation:", sd)
median = np.median(flies[0])
print("Median:", median)




Initial Population:
Min: -4.9936833
Max: 4.964795
Range: 9.958479
Standard Deviation: 0.1878362
Median: 0.0


In [14]:
import numpy as np

def disturbance_threashold_mechanism(delta, disturbance_counter, patience):
    if delta == 1 and disturbance_counter > patience:
        return 0, 0 #delta, disturbance_counter
    elif delta != 1 and disturbance_counter > patience:
        delta = (delta + np.random.uniform(high=0.5)) #add rand number between 0-0.5
        if delta > 1:
            return 1, 0
        else:
            return delta, 0
    else:
        return delta, disturbance_counter

In [ ]:
def DFO(start_flies):
    """
    DFO with dynamic delta control and detailed fitness reporting
    """
    X = np.array(start_flies)     
    fitness = np.zeros(N)         
    #flies_history = [deque(maxlen=5) for _ in range(N)]
    disturbance_counter = 0
    fitness_history = []
    delta_history = []
    delta = 1
    
    for i in range(N):
        #fitness[i] = f(X[i], flies_history[i])
        fitness[i] = f(X[i])


    # Find initial best fly
    s = np.argmin(fitness)
    
    # Main DFO loop
    for itr in range(maxIterations):
        print(f"\n{'='*50}")
        print(f"Iteration: {itr}")
        print(f"Current delta: {delta:.6f}")
        print(f"disturbance_counter: {disturbance_counter}")

        for i in range(N):
            if i == s:
                print(f"Fly {i} (Best fly) fitness = {fitness[i]:.6f}")
                continue

            left = (i-1) % N
            right = (i+1) % N
            bNeighbour = right if fitness[right] < fitness[left] else left

            

            U = np.random.uniform(0, 1, D)
            R = np.random.uniform(0, 1, D)
            X[i] = np.where(R < delta,
                           np.random.normal(loc=X[bNeighbour], scale=2*sd),
                           X[bNeighbour] + U * (X[s] - X[i]))

            #new_fitness = f(X[i], flies_history[i])

            #print(f"\nFly {i} after update:")
            #print(f"First 5 dimensions: {X[i][:5]}")
            
        

            
            fitness[i] = f(X[i])
            
            
            
        new_s = np.argmin(fitness)
        
        if new_s <= s:
            disturbance_counter += 1
            delta, disturbance_counter = disturbance_threashold_mechanism(delta, disturbance_counter, patience=10)
        s = new_s
            

        fitness_history.append(fitness[s])
        delta_history.append(delta)
        
        print(f"\nEnd of iteration summary:")
        print(f"Best fly: {s} with fitness = {fitness[s]:.6f}")
        print(f"All flies fitness values:")
        for i in range(N):
            print(f"Fly {i}: {fitness[i]:.6f}")

    return X[s], fitness_history, delta_history

In [ ]:

best_solution, fitness_history, delta_history = DFO(flies)


In [15]:
np.save('realbest_fly_weights.npy', best_solution)

In [16]:
newweihgts = np.load('realbest_fly_weights.npy')

In [17]:
#Saving fine tuned weights

cloned_model = q_network
    # Reshape fly's weights to match the model's weight shape
reshaped_fly = reshape_to_model_weights(newweihgts)
    
    # Load the reshaped weights into the cloned model
cloned_model.load_state_dict(dict(zip([name for name, _ in cloned_model.named_parameters()], reshaped_fly)))


# save the model as dfoFineTuned
torch.save(cloned_model.state_dict(), '1dfoFineTuned.pt')


In [ ]:
len(fitness_history)

In [1]:
# Plot results after all iterations
import matplotlib.pyplot as plt

# Plot fitness[s]
plt.figure(figsize=(10, 5))
plt.plot(fitness_history, label="Fitness (Best Fly)")
plt.xlabel("Iteration")
plt.ylabel("Fitness")
plt.title("Fitness vs Iteration")
plt.legend()
plt.grid()

# Plot delta
plt.figure(figsize=(10, 5))
plt.plot(delta_history, label="Delta", color='orange')
plt.xlabel("Iteration")
plt.ylabel("Delta")
plt.title("Delta vs Iteration")
plt.legend()
plt.grid()

plt.show()

NameError: name 'fitness_history' is not defined

<Figure size 1000x500 with 0 Axes>